In [ ]:
import pandas as pd
import numpy as np

inform_sim_data_path = 'data_with_inform_sim/data_with_inform_sim_LLaVA_v2.csv'
inform_sim_data = pd.read_csv(inform_sim_data_path)
# inform_sim_data.drop(columns=['simulatable'], inplace=True)
inform_sim_data

In [ ]:
"""Here are the steps I used to copy extracted_rationale_pieces
I will then paste it into the human annotated data on google slides"""
# # Drop if column K is empty
# human_annotated_data_50 = pd.read_excel('../results/Human Annotation of LLaVA+ Rationales.xlsx', header=1).iloc[:, :14].dropna(subset=['Annotator'])
# # Calc inform_sim_data's accuracy on the set of human annotated data
# filtered_data_set_50 = inform_sim_data[inform_sim_data['question'].isin(human_annotated_data_50['question'])]
# merged_data_50 = pd.merge(filtered_data_set_50[['question', 'informative', 'simulatable_mask', 'extracted_rationale_pieces']], human_annotated_data_50, on='question')

# # Output a xlsx file containing only column 'extracted_rationale_pieces'
# # For the sentences in the list in one cell, separate them by '\n'
# merged_data_50['extracted_rationale_pieces'] = merged_data_50['extracted_rationale_pieces'].apply(lambda x: x.replace('\', ', '\', \n'))
# # For every sentence in the list, assign a different color. Say the first sentence is red, the second is orange, the third is green, and so on.



# # Save the data to a xlsx file
# merged_data_50[['question', 'extracted_rationale_pieces']].to_excel('TEMP_extracted_rationale_pieces.xlsx', index=False)

In [ ]:
human_annotated_data_path = '../results/Human Annotation of LLaVA+ Rationales.xlsx'
# Up to column N, drop NaN rows
human_annotated_data = pd.read_excel(human_annotated_data_path, header=1).iloc[:, :14].dropna()
human_annotated_data

In [ ]:
len(human_annotated_data)

In [ ]:
filtered_data_set_50 = inform_sim_data[inform_sim_data['question'].isin(human_annotated_data['question'])]
len(filtered_data_set_50)

In [ ]:
merged_data = pd.merge(filtered_data_set_50[['question', 'informative', 'simulatable_mask', 'support_use_pieces', 'support_use_pieces_mask']], human_annotated_data, on='question')
merged_data.head()

In [ ]:
filtered_data_high_human_sim_info = merged_data[(merged_data['Informativeness'] == 1) & (merged_data['Simulatability'] == 1)]
print(f'Number of high human sim info: { len(filtered_data_high_human_sim_info) }')
filtered_data_high_human_sim_info

In [ ]:
filtered_data_high_auto_sim_info = filtered_data_set_50[(filtered_data_set_50['informative'] == True) & (filtered_data_set_50['simulatable_mask'] == True)]
filtered_data_high_auto_sim_info = pd.merge(filtered_data_high_auto_sim_info, human_annotated_data[['question', 'Informativeness', 'Simulatability', 'Visual Fidelity']], on='question')
print(f'Number of high auto sim info: { len(filtered_data_high_auto_sim_info) }')
filtered_data_high_auto_sim_info

In [ ]:
def calc_statistics(data):
    accuracy = np.mean(data['is_correct'])
    avg_simulatability = np.mean(data['simulatable_mask'])
    avg_informativeness = np.mean(data['informative'])
    percentage_low_human_fidelity = np.mean((data['Visual Fidelity'] == 0) | (data['Visual Fidelity'] == -1))
    print(f'Accuracy: {accuracy}')
    print(f'Average Simulatability: {avg_simulatability}')
    print(f'Average Informativeness: {avg_informativeness}')
    print(f'Percentage of low human fidelity: {percentage_low_human_fidelity}')
    print('---------------------------------')

In [ ]:
for data in [merged_data, filtered_data_high_human_sim_info, filtered_data_high_auto_sim_info]:
    calc_statistics(data)

In [ ]:
%pip install seaborn scikit-learn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Prepare the data
merged_data['Human_Informativeness'] = merged_data['Informativeness'].apply(lambda x: 1 if x == 1 else 0)
merged_data['Human_Simulatability'] = merged_data['Simulatability'].apply(lambda x: 1 if x == 1 else 0)

# Function to reorder confusion matrix
def reorder_confusion_matrix(conf_matrix):
    return conf_matrix[::-1, ::-1]

# Create confusion matrices
conf_matrix_informativeness = confusion_matrix(merged_data['Human_Informativeness'], merged_data['informative'])
conf_matrix_simulatability = confusion_matrix(merged_data['Human_Simulatability'], merged_data['simulatable'])

# Reorder the confusion matrices
conf_matrix_informativeness_reordered = reorder_confusion_matrix(conf_matrix_informativeness)
conf_matrix_simulatability_reordered = reorder_confusion_matrix(conf_matrix_simulatability)

# Plot the confusion matrices
fig, ax = plt.subplots(1, 2, figsize=(10, 4.5))

sns.heatmap(conf_matrix_informativeness_reordered, annot=True, fmt='d', cmap='Blues', ax=ax[0])
ax[0].set_title('Informativeness Confusion Matrix')
ax[0].set_xlabel('Automatic')
ax[0].set_ylabel('Human')
ax[0].set_xticklabels(['1', '0'])
ax[0].set_yticklabels(['1', '0'])

sns.heatmap(conf_matrix_simulatability_reordered, annot=True, fmt='d', cmap='Blues', ax=ax[1])
ax[1].set_title('Simulatability Confusion Matrix')
ax[1].set_xlabel('Automatic')
ax[1].set_ylabel('Human')
ax[1].set_xticklabels(['1', '0'])
ax[1].set_yticklabels(['1', '0'])

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Prepare the data: 1-->1, 0/-1-->0
merged_data['Human_Simulatability'] = merged_data['Simulatability'].apply(lambda x: 1 if x == 1 else 0)

# Function to reorder confusion matrix
def reorder_confusion_matrix(conf_matrix):
    return conf_matrix[::-1, ::-1]

# Create confusion matrices
conf_matrix_simulatability_mask = reorder_confusion_matrix(confusion_matrix(merged_data['Human_Simulatability'], merged_data['simulatable_mask']))
conf_matrix_simulatability_pieces = reorder_confusion_matrix(confusion_matrix(merged_data['Human_Simulatability'], merged_data['support_use_pieces']))
conf_matrix_simulatability_pieces_mask = reorder_confusion_matrix(confusion_matrix(merged_data['Human_Simulatability'], merged_data['support_use_pieces_mask']))

# Plot the confusion matrices
fig, ax = plt.subplots(1, 3, figsize=(15, 4.5))

for i in range(3):
    conf_m_name = [conf_matrix_simulatability_mask, conf_matrix_simulatability_pieces, conf_matrix_simulatability_pieces_mask][i]
    sns.heatmap(conf_m_name, annot=True, fmt='d', cmap='Blues', ax=ax[i])
    ax[i].set_title(['Simulatability Mask Confusion Matrix', 'Simulatability Pieces Confusion Matrix', 'Simulatability Pieces Mask Confusion Matrix'][i])
    ax[i].set_xlabel('Automatic')
    ax[i].set_ylabel('Human')
    ax[i].set_xticklabels(['1', '0'])
    ax[i].set_yticklabels(['1', '0'])

plt.tight_layout()
plt.show()